In [ ]:
import requests
import pandas as pd
from io import StringIO

url = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/all/USW00012839.dly"
response = requests.get(url)

if response.status_code != 200:
    raise Exception("Failed to download data")

data = response.text

# fixed width file parser for GHCN-D format
records = []
for line in data.splitlines():
    station = line[0:11]
    year = int(line[11:15])
    month = int(line[15:17])
    element = line[17:21]

    for day in range(31):
        value = line[21 + day * 8 : 26 + day * 8].strip()
        if value == "-9999" or not value:
            continue
        # NOAA stores tenths of units
        value = float(value) / 10.0
        date = f"{year}-{month:02d}-{day + 1:02d}"
        records.append((station, date, element, value))

df = pd.DataFrame(records, columns=["STATION", "DATE", "ELEMENT", "VALUE"])

# pivot so each element (TMAX, TMIN, PRCP,) is its own col
df = df.pivot_table(index="DATE", columns="ELEMENT", values="VALUE", aggfunc="first").reset_index()

df["DATE"] = pd.to_datetime(df["DATE"])

# filter last 5 yrs
# df = df[df["DATE"] >= "2023-01-01"]
df = df[(df["DATE"] >= "2017-01-01") & (df["DATE"] <= "2024-12-31")]

print(df.head())
print(df.tail())
print(df.shape)
print(df.columns.tolist())


ELEMENT       DATE  ACMH  ACSH  ADPT    ASLP    ASTP  AWBT  AWND  FMTM  PGTM  \
25203   2017-01-01   NaN   NaN  18.9  1021.3  1020.0  21.1   4.4   NaN   NaN   
25204   2017-01-02   NaN   NaN  21.7  1020.7  1019.6  23.3   5.5   NaN   NaN   
25205   2017-01-03   NaN   NaN  21.7  1016.3  1015.6  22.8   3.4   NaN   NaN   
25206   2017-01-04   NaN   NaN  21.7  1012.9  1011.9  22.8   2.7   NaN   NaN   
25207   2017-01-05   NaN   NaN  15.0  1013.5  1011.9  18.3   1.7   NaN   NaN   

ELEMENT  ...  WT07  WT08  WT09  WT10  WT11  WT13  WT14  WT16  WT18  WT21  
25203    ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
25204    ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
25205    ...   NaN   0.1   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
25206    ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
25207    ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[5 rows x 47 columns]
ELEMENT       DATE  ACMH  ACSH  ADPT    ASLP  

In [ ]:
import numpy as np

# identify all wt columns
wt_cols = [col for col in df.columns if col.startswith("WT")]

# fill NaN with 0
df[wt_cols] = df[wt_cols].fillna(0)

# combine all wt columns into a single vector col
df['WT_VECTOR'] = df[wt_cols].apply(lambda row: row.values.astype(float), axis=1)

# drop individul wt cols
df = df.drop(columns=wt_cols)

# drop any wind direction cols
df = df.drop(columns=["SNWD","SNOW","ACMH", "ACSH", "WDF1", "WDF2", "WDF5", "WDFG", "RHMN", "RHMX", "FMTM", "PGTM", "TOBS", "TSUN", "WESD", "WSF1", "WSFG", "PSUN"])

df.head()

ELEMENT,DATE,ADPT,ASLP,ASTP,AWBT,AWND,PRCP,RHAV,TAVG,TMAX,TMIN,WSF2,WSF5,WT_VECTOR
25203,2017-01-01,18.9,1021.3,1020.0,21.1,4.4,0.0,6.9,24.6,28.3,21.7,8.9,12.5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
25204,2017-01-02,21.7,1020.7,1019.6,23.3,5.5,0.0,7.5,26.5,29.4,25.0,10.3,13.9,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
25205,2017-01-03,21.7,1016.3,1015.6,22.8,3.4,17.5,7.9,26.4,30.0,22.8,7.6,10.7,"[0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.1, 0.0, ..."
25206,2017-01-04,21.7,1012.9,1011.9,22.8,2.7,4.1,8.2,24.5,28.3,22.8,5.4,7.6,"[0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
25207,2017-01-05,15.0,1013.5,1011.9,18.3,1.7,0.0,6.2,23.5,27.8,18.3,5.4,8.5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
df.tail()
df.to_csv("miami_weather.csv", index=False)